# Stratospheric water vapour

Requires methane forcing, so let's calculate all GHGs.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pl

from fair21.constants.gases import molwt, burden_per_emission, lifetime
from fair21.defaults.gases import (
    partition_fraction,
    pre_industrial_concentration,
    natural_emissions_adjustment,
    iirf_0,
    iirf_cumulative,
    iirf_temperature,
    iirf_airborne,
    iirf_horizon
)
from fair21.defaults import gas_list
from fair21.defaults.forcing import tropospheric_adjustment
from fair21.gas_cycle import (
    calculate_g,
    calculate_alpha,
    step_concentration_1box,
    step_concentration_co2
)
from fair21.forcing.ghg import meinshausen, linear as linear_forcing
from fair21.forcing.stratospheric_water_vapour import linear as linear_stwv

In [ ]:
# grab emissions
emissions = {}
df = pd.read_csv('../data/rcmip/rcmip-emissions-annual-means-v5-1-0.csv')
for gas in gas_list:
    gas_rcmip_name = gas.replace("-", "")
    emissions[gas] = df.loc[
        (df['Scenario']=='ssp585') & (df['Variable'].str.endswith("|"+gas_rcmip_name)) & (df['Region']=='World'), '1750':
    ].interpolate(axis=1).values.squeeze()

# CO2 and N2O units need to behave
emissions["CO2"] = emissions["CO2"] / 1000
emissions["N2O"] = emissions["N2O"] / 1000

In [ ]:
# grab indicative temperature projections
df = pd.read_csv('../data/rcmip-phase2/rcmip-phase2-fair162-ssp585-mean-temperature.csv')
ssp585_temperature_rfmip = df['temperature'].values

In [ ]:
# initialise dicts for outputs. These will eventually be moved to the forward model
concentration = {}
airborne_emissions = {}
cumulative_emissions = {}
effective_radiative_forcing = {}

n_timesteps = len(emissions[gas])

In [ ]:
concentration['CO2'] = np.ones(n_timesteps) * np.nan

co2_boxes = 0
airborne_emissions = 0
cumulative_emissions['CO2'] = np.cumsum(emissions['CO2'])

# g1 = np.sum(partition_fraction['CO2'] * lifetime['CO2'] * (1 - (1 + iirf_horizon/lifetime['CO2']) * np.exp(-iirf_horizon/lifetime['CO2'])))
# g0 = 1/(np.sinh(np.sum(partition_fraction['CO2']*lifetime['CO2']*(1 - np.exp(-iirf_horizon/lifetime['CO2'])) , axis=-1)/g1))
g0, g1 = calculate_g(lifetime['CO2'], partition_fraction=partition_fraction['CO2'])

for i in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions['CO2'][i],
        airborne_emissions,
        ssp585_temperature_rfmip[i],
        iirf_0["CO2"],
        iirf_cumulative["CO2"],
        iirf_temperature["CO2"],
        iirf_airborne["CO2"],
        g0,
        g1,
    )
    concentration['CO2'][i], co2_boxes, airborne_emissions = step_concentration_co2(
        emissions['CO2'][i], 
        co2_boxes,
        airborne_emissions, 
        burden_per_emission['CO2'],
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration['CO2'],
        timestep=1,
    )

In [ ]:
concentration['CH4'] = np.ones(n_timesteps) * np.nan

ch4_boxes = 0
airborne_emissions = 0
concentration['CH4'][0] = pre_industrial_concentration['CH4']
cumulative_emissions['CH4'] = np.cumsum(emissions['CH4']) # cumulative emissions are not contributing to methane lifetime, but we'll track them in case future research makes them relevant

g0, g1 = calculate_g(lifetime['CH4'])

ch4_lifetime = np.ones(n_timesteps) * np.nan

for i in range(n_timesteps):
    alpha_lifetime = calculate_alpha(
        cumulative_emissions['CH4'][i],
        airborne_emissions,
        ssp585_temperature_rfmip[i],
        iirf_0["CH4"],
        iirf_cumulative["CH4"],
        iirf_temperature["CH4"],
        iirf_airborne["CH4"],
        g0,
        g1,
    )
    
    ch4_lifetime[i] = alpha_lifetime * lifetime['CH4']
    
    concentration['CH4'][i], airborne_emissions = step_concentration_1box(
        emissions['CH4'][i], 
        airborne_emissions, 
        lifetime['CH4'],
        burden_per_emission['CH4'],
        alpha_lifetime=alpha_lifetime,
        pre_industrial_concentration=pre_industrial_concentration['CH4'],
        timestep=1,
    )

In [ ]:
for gas in gas_list:
    if gas in ['CH4', 'CO2']:
        continue  # done above and slightly different to rest!
    
    concentration[gas] = np.ones(n_timesteps) * np.nan

    airborne_emissions = 0

    for i in range(n_timesteps):
        concentration[gas][i], airborne_emissions = step_concentration_1box(
            emissions[gas][i], 
            airborne_emissions,
            lifetime[gas],
            burden_per_emission[gas],
            pre_industrial_concentration=pre_industrial_concentration[gas],
            timestep=1,
            natural_emissions_adjustment=natural_emissions_adjustment[gas]
        )

In [ ]:
_, effective_radiative_forcing = meinshausen(
    concentration,
    pre_industrial_concentration,
    tropospheric_adjustment
)

In [ ]:
concentration

In [ ]:
_, effective_radiative_forcing_minor = linear_forcing(
    concentration
)
effective_radiative_forcing.update(effective_radiative_forcing_minor)

In [ ]:
fig, ax = pl.subplots(6, 8, figsize=(16,16))
for igas, gas in enumerate(gas_list):
    iy = igas % 8
    ix = igas // 8
    ax[ix, iy].plot(np.arange(1750.5, 2501), effective_radiative_forcing[gas], label='FaIR 2.1')
    #ax[ix, iy].plot(np.arange(1750.5, 2501), concentration_rcmip[gas], label='History + MAGICC6')
    ax[ix, iy].set_title(gas)
fig.tight_layout()

In [ ]:
effective_radiative_forcing['Stratospheric water vapour|CH4'] = linear_stwv(
    effective_radiative_forcing
)

In [ ]:
# check to see if direct array input also works.
erf_stwv_numpy = linear_stwv(effective_radiative_forcing['CH4'])

In [ ]:
pl.plot(np.arange(1750.5, 2501), effective_radiative_forcing['Stratospheric water vapour|CH4'])
pl.plot(np.arange(1750.5, 2501), erf_stwv_numpy)